In [ ]:
from src.data_utils.dataset import build_datset_train_val
from src.rnn.gru_prototipes import build_gru_model
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf

In [ ]:
train_dataset, val_dataset = build_datset_train_val(split=0.8, batch_size=256)

In [ ]:
def objective(trial):
    tf.keras.backend.clear_session()
    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=get_predefine_callbacks(model_name="v1"))
    print(model.summary())

    val_loss = model.evaluate(val_dataset)[0]
    return  val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, gc_after_trial=True, show_progress_bar=True)

In [ ]:
tf.keras.backend.clear_session()

trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "v1"

    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=5000, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation loss: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Score in Validation: {model.evaluate(val_dataset)}")